In [1]:
import gradio as gr
import os
import joblib


c:\Users\unaif\anaconda3\envs\projecto\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.chdir("../src")
from tool_preprocess import State_enc, product_encoder, sub_Issue_enc, sub_product_encoder,  Issue_enc, Company_response_enc, Time_enco,Company_enc, preprocesing_function_paraforest, week_enc
preprocesador = joblib.load("preprocesador_red.pkl")

c:\Users\unaif\anaconda3\envs\projecto\Lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\unaif\anaconda3\envs\projecto\Lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\unaif\anaconda3\envs\projecto\Lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\unaif\anaconda3\envs\projecto\Lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d

In [3]:
Time_enco

LabelEncoder()

In [4]:
# Aplicacion
import pandas as pd
import pickle
import os

import os
import joblib

os.chdir("../models")
with open('modelo_timely_tree_sin_comp.pkl', 'rb') as f:
    modelo_timely = pickle.load(f)

In [5]:
import dill
import __main__
import sys
sys.path.append(os.path.abspath("../src"))  # ajusta según tu estructura
def convert_to_str(X):
    return X.astype(str)
__main__.convert_to_str = convert_to_str



from tensorflow import keras
modelo_dispute = keras.models.load_model("modelo_dispute_red.keras")


In [6]:
product_encoder.classes_.tolist()

['Bank account or service',
 'Consumer loan',
 'Credit card',
 'Credit reporting',
 'Debt collection',
 'Money transfers',
 'Mortgage',
 'Other financial service',
 'Payday loan',
 'Prepaid card',
 'Student loan',
 'Unknown or not specified']

In [7]:
product_encoder.transform(pd.DataFrame(product_encoder.classes_.tolist()))

c:\Users\unaif\anaconda3\envs\projecto\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [8]:
import pickle
import joblib
os.chdir("../models")
#saco las pipelines /modelos


modelo_timely=joblib.load("modelo_timely_tree_sin_comp.pkl")
def predict_timely (    
    Product:str,
    Subproduct:str,
    Issue:str,
    Subissue:str,
    State:str,
    ):
    
    #State por defecto Unknown or not specified
    #Subproduct por defecto Unknown or not specified
    #Subissue por defecto Unknown or not specified
    #Product por defecto other financial services

    #Yes=1, No= 0


    
# 
    features=pd.DataFrame({"Product":[Product],
                           "Sub-product": [Subproduct],	
            "Issue":[Issue],	
            "Sub-issue":[Subissue],	
            "State":[State],	
            })

    display(features)
    features["Product"] = product_encoder.transform(features["Product"])
    features["Sub-product"] = sub_product_encoder.transform(features["Sub-product"])
    features["Issue"] = Issue_enc.transform(features["Issue"])
    features["Sub-issue"] = sub_Issue_enc.transform(features["Sub-issue"])
    features["State"] = State_enc.transform(features["State"])
    
    
    display(features)
    pred=(float(modelo_timely.predict_proba(features)[:,1])>=0.5)
    prob= float(modelo_timely.predict_proba(features)[:,1])

    if int(pred)==1:
        response="Yes"
    else:
        response="No"

    return {"response01": int(pred), "timeprob":prob, "response":response
            }



In [9]:
pd.DataFrame(["Yes"])

,0
0,Yes


In [10]:
int(Time_enco.transform(pd.DataFrame(["Yes"]))[0])

c:\Users\unaif\anaconda3\envs\projecto\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


1

In [11]:
def predict_dispute (
    Product:str, 
    Subproduct:str, 
    Issue:str, 
    Subissue:str, 
    State:str,

    Company:str,

    Companyresponse:str, 
    weekday:str,
    timely=None
    ):

    """
    Funcion que predice si el cliente disputara o no.
    Si no se da timely response lo predice y da los reslutado
    Devuelve:
        "response":Yes/No
        "response01": 1/0
        "prob": %
        timely:
            "response":Yes/No
            "response01": 1/0
            "prob": %

    """
    

    if timely == None:
        timely_res = (predict_timely(Issue=Issue,Subissue=Subissue ,  Product=Product, Subproduct=Subproduct , State=State ))
        timely=timely_res["response"]
    else:
        timely_res='Not calculated'

    print(timely_res)
    
    features=pd.DataFrame({

            "Product":[Product],
            "Sub-product": [Subproduct],	
            "Issue":[Issue],	
            "Sub-issue":[Subissue],	
            "State":[State],	
            "Company":[Company],
            "Company response":[Companyresponse],
            "Timely response?": [timely],
            
            
            
            "weekday":[weekday],})
    


    features_pro=preprocesador.transform(features)

    #pred=modelo_dispute.predict(features_pro)[0]
    pred=(modelo_dispute.predict(features_pro) >= 0.5).astype(int)
    if pred== 1:
        res="Yes"
    else :
        res="No"
    prob=modelo_dispute.predict(features_pro)

    return {"response":res, "response01": int(pred), "prob": float(prob), "timely":timely_res}

In [12]:
product_encoder.transform(pd.DataFrame(["Debt collection"]))

c:\Users\unaif\anaconda3\envs\projecto\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


array([4])

In [13]:
predict_dispute("Debt collection","Unknown or not specified","Communication tactics",	"Frequent or repeated calls", "TX", "Premium Asset Services, LLC", "In progress","Yes", "Monday" )

Not calculated
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


C:\Users\unaif\AppData\Local\Temp\ipykernel_4088\2715629170.py:65: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return {"response":res, "response01": int(pred), "prob": float(prob), "timely":timely_res}


{'response': 'Yes',
 'response01': 1,
 'prob': 0.9999696612358093,
 'timely': 'Not calculated'}

In [14]:
iface = gr.Interface(
    fn=predict_dispute,
    inputs=[
        gr.Dropdown(choices=product_encoder.classes_.tolist(), value='Unknown or not specified', label="Product"),
        gr.Dropdown(choices=sub_product_encoder.classes_.tolist(), value='Unknown or not specified', label="Sub-product"),
        gr.Dropdown(choices=Issue_enc.classes_.tolist(), value='Unknown or not specified', label="Issue"),
        gr.Dropdown(choices=sub_Issue_enc.classes_.tolist(), value='Unknown or not specified', label="Sub-issue"),
        
        gr.Dropdown(choices=State_enc.classes_.tolist(), value='Unknown or not specified', label="State"),
        gr.Dropdown(choices=Company_enc.classes_.tolist(), value='Unknown or not specified', label="Company"),
        gr.Dropdown(choices=Company_response_enc.classes_.tolist(), value='Unknown or not specified', label="Company response"),
        gr.Dropdown(choices=week_enc.classes_.tolist(),  label="weekday"),
        gr.Dropdown(choices=["Yes", "No", None], value=None, label="Timely response?"),
       
        
        
    ],
    outputs="json",
    title="Predicción Timely",
    description="Introduce los parámetros para obtener la predicción."
)
if __name__ == "__main__":
    iface.launch()

c:\Users\unaif\anaconda3\envs\projecto\Lib\site-packages\gradio\components\dropdown.py:230: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include: Unknown or not specified or set allow_custom_value=True.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [ ]:
"False "

'False '

,Product,Sub-product,Issue,Sub-issue,State
0,Unknown or not specified,Unknown or not specified,Unknown or not specified,Unknown or not specified,Unknown or not specified


,Product,Sub-product,Issue,Sub-issue,State
0,11,39,86,45,52


{'response01': 1, 'timeprob': 0.9333333333333333, 'response': 'Yes'}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


C:\Users\unaif\AppData\Local\Temp\ipykernel_4088\2486008902.py:42: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred=(float(modelo_timely.predict_proba(features)[:,1])>=0.5)
C:\Users\unaif\AppData\Local\Temp\ipykernel_4088\2486008902.py:43: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  prob= float(modelo_timely.predict_proba(features)[:,1])
C:\Users\unaif\AppData\Local\Temp\ipykernel_4088\2715629170.py:65: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return {"res

...

C:\Users\unaif\AppData\Local\Temp\ipykernel_4088\2486008902.py:42: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred=(float(modelo_timely.predict_proba(features)[:,1])>=0.5)
C:\Users\unaif\AppData\Local\Temp\ipykernel_4088\2486008902.py:43: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  prob= float(modelo_timely.predict_proba(features)[:,1])
C:\Users\unaif\AppData\Local\Temp\ipykernel_4088\2715629170.py:65: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return {"res

Not calculated
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


C:\Users\unaif\AppData\Local\Temp\ipykernel_4088\2715629170.py:65: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return {"response":res, "response01": int(pred), "prob": float(prob), "timely":timely_res}


Not calculated
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


C:\Users\unaif\AppData\Local\Temp\ipykernel_4088\2715629170.py:65: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return {"response":res, "response01": int(pred), "prob": float(prob), "timely":timely_res}
